# Evaluate Fine-Tuned Llama Using DeepEval

In [1]:
#!srun --gres=gpu:1 --mem=48G --account=fta_boot --qos=fta_boot --partition=a40 --pty bash
# !scancel --user=ws_aabboud

In [43]:
# !pip install evaluate
# !pip install pandas
# !pip install numpy
# !pip install absl-py
# !pip install rouge-score
# !pip install python-dotenv

## 1. Fine-Tuning Llama 3

In [44]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,

    get_peft_model,
)
import ast
import re
import time
import os, torch
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import SFTTrainer, setup_chat_format
from dotenv import load_dotenv 

from deepeval.integrations.hugging_face import DeepEvalHuggingFaceCallback
from deepeval.metrics import AnswerRelevancyMetric, GEval
from deepeval import evaluate
from deepeval.dataset import EvaluationDataset, Golden
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from deepeval.models.base_model import DeepEvalBaseLLM

# from deepeval_gemini import GoogleVertexAI

import torch
import time
import evaluate 
import pandas as pd
import numpy as np
from transformers.generation import GenerationConfig
import os


In [55]:
home=os.path.expanduser('~')
os.chdir(f'{home}/finetuning-and-alignment/Deloitte/finetuned/')
# os.getcwd()
load_dotenv(override=True)
## OpenAI
os.environ["OPENAI_API_KEY"] =os.getenv('OPENAI_API_KEY')

True

### Login to HuggingFace Hub

In [2]:
# from huggingface_hub import login
# from huggingface_hub import interpreter_login

# interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: fineGrained).
Your token has been saved to /h/ws_jinxiao/.cache/huggingface/token
Login successful


# Load your Model and Create Actual Output

In [2]:
os.getcwd()

'/fs01/home/ws_jinxiao/llm_finetuning/Deloitte/finetuned'

In [56]:
os.chdir("../../../../../../")
os.getcwd()

'/'

In [40]:
shared_space="/fs01/projects/fta_teams/deloitte"
base_model = "/fs01/model-weights/Meta-Llama-3-8B-Instruct"
dataset_name = "heliosbrahma/mental_health_conversational_dataset"

# new_model = "/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/models/llama-3-8b-chat-doctor"
merged_model= f"{shared_space}/merged_models/llama-3-8b-chat-doctor-merged-shyana-v1"
adapter_model=f"{shared_space}/adapters/llama-3-8b-finetune-shyana-fulldatav1"

In [18]:
# base_model = "/fs01/model-weights/Mistral-7B-v0.1" # pretend this to be the fine-tuned model

### Set the data type and attention implementation

In [22]:
torch_dtype = torch.float16
attn_implementation = "eager"

## Loading the Model

In [23]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Load Tokenizer

In [24]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Adding the adapter to the layer

In [9]:
# # LoRA config
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
# )
# model = get_peft_model(model, peft_config)

## Loading Dataset

In [25]:
dataset_name = "heliosbrahma/mental_health_conversational_dataset"

In [26]:
"""
This function is to convert the dataset input to a LLLM acceptable input, 
then this acceptable input will be further converted into chat format by huggingface's apply_chat_template
"""

def extract_message(text, tag):
    """
    Extracts a message from the given text based on the specified tag.
    
    Parameters:
    text (str): The text containing the messages.
    tag (str): The tag to search for, either '<<<HUMAN>>>: ' or '<<<ASSISTANT>>>: '.
    
    Returns:
    str: The extracted message, or None if the tag is not found.
    """
    try:
        start_tag = f'<<<{tag}>>>: '
        end_tag = ' <<<'
        
        start_index = text.find(start_tag)
        if start_index == -1:
            return None
        
        start_index += len(start_tag)
        end_index = text.find(end_tag, start_index)
        
        if end_index == -1:
            # If end_tag not found, take the rest of the text
            end_index = len(text)
        
        return text[start_index:end_index].strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [27]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(100)) # Only use 100 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": extract_message(row["text"], 'HUMAN')},
               {"role": "assistant", "content":  extract_message(row["text"], 'ASSISTANT')}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

Using the latest cached version of the dataset since heliosbrahma/mental_health_conversational_dataset couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /h/ws_aabboud/.cache/huggingface/datasets/heliosbrahma___mental_health_conversational_dataset/default/0.0.0/fc8f9432d09ed6e54a3fc799daf7ff0a69a5d293 (last modified on Tue Jul 23 13:56:54 2024).


'<|im_start|>user\nWhat are the side effects of medication?<|im_end|>\n<|im_start|>assistant\nLike other medication, psychiatric medication has its own set of side effects like Drowsiness, Restlessness, Dizziness, Dry mouth, Constipation, Nausea, and Vomiting.\nIt is usually because of the body getting used to medication. It normally takes a month for the body to get used to these drugs. If you are feeling any of the above symptoms, call your doctor immediately.<|im_end|>\n'

### Split the Dataset

In [28]:
dataset = dataset.train_test_split(test_size=0.1)

In [59]:
dataset


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 90
    })
    test: Dataset({
        features: ['text'],
        num_rows: 10
    })
})

In [60]:
train_dataset = dataset['train']
val_dataset = dataset['test']

In [29]:
os.getcwd()

'/fs01/home/ws_aabboud/finetuning-and-alignment'

## Produce Model Actual Output

In [30]:
# function to extract a prompt like test input

def extract_test_set_input_for_models(text):
    # Define the regex pattern to match the assistant's output
    pattern = r"(<\|im_start\|>user\n.*assistant\n)" 
    
    # Use re.search to find the match
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract the matched text
        return match.group(1).strip()
    else:
        return None

In [31]:
extracted_test_input_question = extract_test_set_input_for_models(dataset["test"]["text"][0])

# adding the \n breaker in the end, because the extraction is not able to extract the \n
extracted_test_input_question = f"{extracted_test_input_question}\n"

In [32]:
extracted_test_input_question

'<|im_start|>user\nHow can I find help for an alcohol or drug use problem?<|im_end|>\n<|im_start|>assistant\n'

In [33]:
# Usually we need 
# messages = [{"role": "user", "content": "Hello doctor, I have bad insomnia. How do I get rid of it?"}]
# prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

# I made a function extract_test_set_input_for_models to let you use it directly to the testset to produce what is similar to prompt
outputs = pipe(extracted_test_input_question, max_new_tokens=256, do_sample=True, temperature=0.3, top_k=50, top_p=0.95) # you could change the max_new_token accordingly 
print(outputs[0]["generated_text"])

<|im_start|>user
How can I find help for an alcohol or drug use problem?<|im_end|>
<|im_start|>assistant
                                                                                                                                                                                                                                                                


In [34]:
outputs

[{'generated_text': '<|im_start|>user\nHow can I find help for an alcohol or drug use problem?<|im_end|>\n<|im_start|>assistant\n                                                                                                                                                                                                                                                                '}]

In [36]:
import re

def extract_assistant_output(text):
    # Define the regex pattern to match the assistant's output
    pattern = r"<\|im_start\|>assistant(.*)"
    
    # Use re.search to find the match
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        # Extract the matched text
        return match.group(1).strip()
    else:
        return None

In [61]:
import re

def extract_user_query(text):
    # Define the regex pattern to capture the text between the 'user\n' and '\nassistant\n'
    pattern = r'(<\|im_start\|>user\n.*?<\|im_end\|>)'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the captured group (the user's query)
    if match:
        return match.group(1).strip()
    else:
        return None
goldens = []
for sentence in dataset["test"]["text"]:
    cleansed_input = extract_user_query(sentence)
    golden_input = Golden(input=cleansed_input, context=[""])
    goldens.append(golden_input)

In [62]:
# Use the function to extract the assistant's text
extracted_text = extract_assistant_output(outputs[0]["generated_text"])
print(extracted_text)

# 2. Using DeepEval During Fine-tuning (Running into errors) regardless of using Huggingface model or Gemini

### create a DeepEval HuggingFace Callback (FAILED)

In [30]:
from transformers import AutoModel



# Jingy Code
# eval_model_id ="TheBloke/Mistral-7B-v0.1-GGUF"
# eval_file_name = "mistral-7b-v0.1.Q5_K_M.gguf"
# eval_tokenizer = AutoTokenizer.from_pretrained(eval_model_id, gguf_file=eval_file_name)
# eval_model = AutoModelForCausalLM.from_pretrained(eval_model_id, gguf_file=eval_file_name)

In [41]:
# gguf_model= f"{shared_space}/gguf_models/llama-3-8b-chat-doctor-merged-shyana-q8-v1.gguf"
# eval_tokenizer = AutoTokenizer.from_pretrained("llama-3-8b-chat-doctor-merged-shyana-q8-v1", gguf_file=gguf_model)
# eval_model = AutoModelForCausalLM.from_pretrained("llama-3-8b-chat-doctor-merged-shyana-q8-v1", gguf_file=gguf_model)
eval_tokenizer = AutoTokenizer.from_pretrained(merged_model)
eval_model = AutoModelForCausalLM.from_pretrained(merged_model, device_map="auto")
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.models.base_model import DeepEvalBaseLLM

class Mistral7B(DeepEvalBaseLLM):
    def __init__(
        self,
        model,
        tokenizer
    ):
        self.model = model
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        device = "cuda" # the device to load the model onto

        model_inputs = self.tokenizer([prompt], return_tensors="pt").to(device)
        model.to(device)

        generated_ids = model.generate(**model_inputs, 
                                       max_length=1000, 
                                       # max_new_tokens=1000, 
                                       do_sample=True)
        return self.tokenizer.batch_decode(generated_ids)[0]

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Mistral 7B"

mistral_7b = Mistral7B(model=eval_model, tokenizer=eval_tokenizer)
print(mistral_7b.generate("Write me a joke"))

In [46]:
# use evalatuion metric
# answer_relevancy_metric = AnswerRelevancyMetric(model=mistral_7b)


In [57]:
## OpenAI
os.environ["OPENAI_API_KEY"] =os.getenv('OPENAI_API_KEY')
# use evalatuion metric
answer_relevancy_metric = AnswerRelevancyMetric()

In [63]:
eval_dataset = EvaluationDataset(goldens=goldens)
eval_dataset

EvaluationDataset(test_cases=[], goldens=[Golden(input='<|im_start|>user\nHow can I find help for an alcohol or drug use problem?<|im_end|>', actual_output=None, expected_output=None, context=[''], retrieval_context=None, additional_metadata=None, comments=None, source_file=None), Golden(input='<|im_start|>user\nHow common are mental illnesses?<|im_end|>', actual_output=None, expected_output=None, context=[''], retrieval_context=None, additional_metadata=None, comments=None, source_file=None), Golden(input='<|im_start|>user\nAre there any ethnic/racial groups that more likely to have mental illnesses?<|im_end|>', actual_output=None, expected_output=None, context=[''], retrieval_context=None, additional_metadata=None, comments=None, source_file=None), Golden(input='<|im_start|>user\nHow much alcohol is considered “too much”?<|im_end|>', actual_output=None, expected_output=None, context=[''], retrieval_context=None, additional_metadata=None, comments=None, source_file=None), Golden(input

In [65]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is relevant based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    # model=mistral_7b
)

In [66]:
tokenizer_args = {
    'max_length': 512,  # Adjust as needed
    'return_tensors': 'pt',
}

In [36]:
deepeval_hugging_face_callback = DeepEvalHuggingFaceCallback(
    evaluation_dataset=eval_dataset,
    # metrics=[correctness_metric],
    metrics=[answer_relevancy_metric],
    trainer=trainer,
    tokenizer_args = tokenizer_args
)

In [37]:
### Add DeepEval Callback
trainer.add_callback(deepeval_hugging_face_callback)

# 3. Evaluate the model after fine-tuning

## Using Deepeval with Gemini (Very Slow if no OpenAI key

In [17]:
os.getcwd()

'/'

In [18]:
# %load_ext autoreload
# %autoreload 2

In [19]:
home=os.path.expanduser('~')
os.chdir(f'{home}/llm_finetuning/Deloitte/finetuned/')
os.getcwd()

'/fs01/home/ws_jinxiao/llm_finetuning/Deloitte/finetuned'

In [32]:
# try Gemini  -- Skip this cell

# Setup Gemini Model
# from langchain_google_genai import ChatGoogleGenerativeAI
# generation_config = {
#     "temperature": 1,
#     "top_p": 0.95,
#     "top_k": 64,
#     "max_output_tokens": 1024,
#     "max_new_tokens":1024,
#     "response_mime_type": "text/plain",
#     }



# gemini_model=ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.7, google_api_key=os.getenv('GEMINI_API_KEY'))

# # initiatialize the  wrapper class
# vertexai_gemini = GoogleVertexAI(model=gemini_model)
# print(vertexai_gemini.generate("Write me a joke"))

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


Why don't scientists trust atoms? 

Because they make up everything! 



## Using OpenAI

firstly create a .env under the same folder
secondly, open terminal in jupyter notebook, run

deepeval set-azure-openai --openai-endpoint=<endpoint> \
    --openai-api-key=<api_key> \
    --deployment-name=<deployment_name> \
    --openai-api-version=<openai_api_version> \
    --model-version=<model_version>


we will need to use openai key

if you don't want to run deepeval command, you can also put those strings into .env file and use os.getenv() to load the enviornment variable

In [ ]:
# from langchain_openai import AzureChatOpenAI
from deepeval.models.base_model import DeepEvalBaseLLM

# class AzureOpenAI(DeepEvalBaseLLM):
#     def __init__(
#         self,
#         model
#     ):
#         self.model = model

#     def load_model(self):
#         return self.model

#     def generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         return chat_model.invoke(prompt).content

#     async def a_generate(self, prompt: str) -> str:
#         chat_model = self.load_model()
#         res = await chat_model.ainvoke(prompt)
#         return res.content

#     def get_model_name(self):
#         return "Custom Azure OpenAI Model"

# # Replace these with real values
# custom_model = AzureChatOpenAI(
#     openai_api_version=openai_api_version,
#     azure_deployment=azure_deployment,
#     azure_endpoint=azure_endpoint,
#     openai_api_key=openai_api_key,
# )
# azure_openai = AzureOpenAI(model=custom_model)
# print(azure_openai.generate("Write me a joke"))

### AnswerRelevancy Metric

In [51]:
answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-3.5-turbo", # change to openai if you are using openai
    include_reason=True
)

In [34]:
# cleanse User input to include only the question

def extract_user_query(text):
    # Define the regex pattern to capture the text between the 'user\n' and '\nassistant\n'
    pattern = r'<\|im_start\|>user(\n.*?)<\|im_end\|>'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the captured group (the user's query)
    if match:
        return match.group(1).strip()
    else:
        return None

In [35]:
# apply the function and append a list of cleansed input

cleansed_test_input = []
for sentence in dataset["test"]["text"]:
    cleansed_test_input.append(extract_user_query(sentence))

print(cleansed_test_input[0])

How does mental health affect physical health?


In [36]:
# function to cleanse the output

def extract_assistant_answer(text):
    # Define the regex pattern to capture the text between the 'user\n' and '\nassistant\n'
    pattern = r'<\|im_start\|>assistant(\n.*?)<\|im_end\|>'
    
    # Search for the pattern in the text
    match = re.search(pattern, text, re.DOTALL)
    
    # If a match is found, return the captured group (the user's query)
    if match:
        return match.group(1).strip()
    else:
        return None

In [37]:
# apply the assistant output

cleansed_assistant_output = []
for sentence in dataset["test"]["text"]:
    cleansed_assistant_output.append(extract_assistant_answer(sentence))

print(cleansed_assistant_output[0])

Mental health and physical health go hand-in-hand. For example, physical exercise can improve your mood and reduce your anxiety, among many other benefits. Similarly, neglecting your mental health can lead to serious health complications, including heart disease, high blood pressure, a weakened immune system, obesity, and more. Depression can cause problems like insomnia and chronic fatigue.


In [38]:
def generate_test_case_for_test_set(test_inputs, actual_outputs):
    test_cases = []
    for test_input, actual_output in zip(test_inputs, actual_outputs):
        test_case = LLMTestCase(
            input=test_input,
            actual_output=actual_output,)
        test_cases.append(test_case)
    return test_cases

In [39]:
test_cases = generate_test_case_for_test_set(cleansed_test_input, cleansed_assistant_output)

In [58]:
test_cases[0]

LLMTestCase(input='How does mental health affect physical health?', actual_output='Mental health and physical health go hand-in-hand. For example, physical exercise can improve your mood and reduce your anxiety, among many other benefits. Similarly, neglecting your mental health can lead to serious health complications, including heart disease, high blood pressure, a weakened immune system, obesity, and more. Depression can cause problems like insomnia and chronic fatigue.', expected_output=None, context=None, retrieval_context=None, additional_metadata=None, comments=None)

In [49]:
test_cases[0].actual_output

'Mental health and physical health go hand-in-hand. For example, physical exercise can improve your mood and reduce your anxiety, among many other benefits. Similarly, neglecting your mental health can lead to serious health complications, including heart disease, high blood pressure, a weakened immune system, obesity, and more. Depression can cause problems like insomnia and chronic fatigue.'

In [ ]:
# due to the quota limit, we will use for loop to evaluate the test case

In [ ]:
metric_name = "answer relevancy"
metric_name_column = [metric_name* len(test_cases)]
test_case_input = []
test_case_actual_output = []
expected_output = []
metric_scores = []
metric_reasonings = []
# columns = ['metric_name', 'model_input', 'model_actual_output', 'expected_output', 'metric_score', 'metric_reasoning']

for test_case in test_cases:
    test_case_input.append(test_case.input)
    test_case_actual_output.append(test_case.actual_output)
    expected_output.append(test_case.expected_output)
    
    answer_relevancy_metric.measure(test_case)
    metric_scores.append(answer_relevancy_metric.score)
    metric_reasonings.append(answer_relevancy_metric.reason)
    time.sleep(60)

# Create a dictionary with the lists
data_dict = {
    'metric_name': metric_name_column,
    'model_input': test_case_input,
    'test_case_actual_output': model_actual_output,
    'expected_output': expected_output,
    'metric_score': metric_scores,
    'metric_reasoning': metric_reasonings,
}

# Create a DataFrame using the dictionary
answer_relevancy_df = pd.DataFrame(data_dict)


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 4.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 8.0 seconds as it 
raised ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)..

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 2.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 4.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 8.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

Retrying langchain_google_genai.chat_models._achat_with_retry.<locals>._achat_with_retry in 16.0 seconds as it 
raised InternalServerError: 500 An internal error has occurred. Please retry or report in 
https://developers.generativeai.google/guide/troubleshooting.

In [ ]:
# answer_relevancy = create_test_case_evaluation_table(test_cases, answer_relevancy_metric, "answer relevancy")

In [ ]:
# evaluate test cases in bulk
# eval_set = evaluate(test_cases[:1], [metric])

In [46]:
# eval_set[0]

TestResult(success=True, metrics_metadata=[MetricMetadata(metric='Answer Relevancy', threshold=0.7, success=True, score=1.0, reason='This is an excellent score!  Keep up the great work!', strict_mode=False, evaluation_model='Vertex AI Model', error=None, evaluation_cost=None)], input='How does mental health affect physical health?', actual_output='Mental health and physical health go hand-in-hand. For example, physical exercise can improve your mood and reduce your anxiety, among many other benefits. Similarly, neglecting your mental health can lead to serious health complications, including heart disease, high blood pressure, a weakened immune system, obesity, and more. Depression can cause problems like insomnia and chronic fatigue.', expected_output=None, context=None, retrieval_context=None)

## Empathy Metric

In [ ]:
empathy_metric = GEval(
    name="Empathy",
    criteria="Determine whether the actual output demonstrates empathy and emotional understanding based on the expected output.",
    evaluation_steps=[
        "Check whether the 'actual output' acknowledges and validates the emotions expressed in the 'expected output'",
        "Assess whether the 'actual output' provides comfort, support, or understanding in response to the emotions expressed",
        "Consider if the 'actual output' uses empathetic language and tone appropriate to the context"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model="gpt-3.5-turbo", # change to openai if you are using openai
)

In [ ]:
# test a single test case
test_case = LLMTestCase(
    input="The dog chased the cat up the tree, who ran up the tree?",
    actual_output="It depends, some might consider the cat, while others might argue the dog.",
    expected_output="The cat."
)

empathy_metric.measure(test_case)
print(empathy_metric.score)
print(empathy_metric.reason)

In [ ]:
# use the same test_cases generated by previous section
metric_name = "empathy"
metric_name_column = [metric_name* len(test_cases)]
test_case_input = []
test_case_actual_output = []
expected_output = []
metric_scores = []
metric_reasonings = []
# columns = ['metric_name', 'model_input', 'model_actual_output', 'expected_output', 'metric_score', 'metric_reasoning']

for test_case in test_cases:
    test_case_input.append(test_case.input)
    test_case_actual_output.append(test_case.actual_output)
    expected_output.append(test_case.expected_output)
    
    empathy_metric.measure(test_case)
    metric_scores.append(empathy_metric.score)
    metric_reasonings.append(empathy_metric.reason)
    time.sleep(60)

# Create a dictionary with the lists
data_dict = {
    'metric_name': metric_name_column,
    'model_input': test_case_input,
    'test_case_actual_output': test_case_actual_output,
    'expected_output': expected_output,
    'metric_score': metric_scores,
    'metric_reasoning': metric_reasonings,
}

# Create a DataFrame using the dictionary
empathy_df = pd.DataFrame(data_dict)

### Combine two metric into one dataframe

In [ ]:
evaluation_df = pd.concat([answer_relevancy_df, empathy_df], ignore_index=True, sort=False)

#### Answer Relevancy Score:

In [ ]:
average_score_for_answer_relevancy = answer_relevancy_df['metric_score'].mean()

print(f"The average score for answer relevancy metric is {average_score_for_answer_relevancy:.2f}")

#### Empathy Score

In [ ]:
average_score_for_empathy = empathy_df['metric_score'].mean()

print(f"The average score for answer empathy metric is {average_score_for_empathy:.2f}")

## BackUp Solution - ROUGE

In [17]:
rouge = evaluate.load('rouge')

In [18]:
dialogue = dataset['test']['text']
human_baseline_summaries = dataset['test']['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogue):
    prompt = f"""
    Summarize the following conversations. 

    {dialogue}

    Summary: """

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)
    
   

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries'])

NameError: name 'dataset' is not defined

In [ ]:
fintuned_model_results = rouge.compute(predictions=actual_outputs, 
                                       references=expected_outputs,
                                      use_aggregator=True,

print(f'Finetuned Model ROUGE scores: \n{fintuned_model_results}\n') 